In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pankajlamgria","key":"1a45498f53b0668c01ddc274770b9cdb"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d manjilkarki/deepfake-and-real-images


Dataset URL: https://www.kaggle.com/datasets/manjilkarki/deepfake-and-real-images
License(s): unknown
 99% 1.67G/1.68G [00:20<00:00, 104MB/s] 
100% 1.68G/1.68G [00:20<00:00, 86.5MB/s]


In [ ]:
! unzip deepfake-and-real-images.zip

Streaming output truncated to the last 5000 lines.
  inflating: Dataset/Validation/Real/real_5499.jpg  
  inflating: Dataset/Validation/Real/real_55.jpg  
  inflating: Dataset/Validation/Real/real_550.jpg  
  inflating: Dataset/Validation/Real/real_5500.jpg  
  inflating: Dataset/Validation/Real/real_5501.jpg  
  inflating: Dataset/Validation/Real/real_5502.jpg  
  inflating: Dataset/Validation/Real/real_5503.jpg  
  inflating: Dataset/Validation/Real/real_5504.jpg  
  inflating: Dataset/Validation/Real/real_5505.jpg  
  inflating: Dataset/Validation/Real/real_5506.jpg  
  inflating: Dataset/Validation/Real/real_5507.jpg  
  inflating: Dataset/Validation/Real/real_5508.jpg  
  inflating: Dataset/Validation/Real/real_5509.jpg  
  inflating: Dataset/Validation/Real/real_551.jpg  
  inflating: Dataset/Validation/Real/real_5510.jpg  
  inflating: Dataset/Validation/Real/real_5511.jpg  
  inflating: Dataset/Validation/Real/real_5512.jpg  
  inflating: Dataset/Validation/Real/real_5513.jpg  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [ ]:
dir=os.listdir("/content/Dataset/Train/Real")
print(dir)

['real_57767.jpg', 'real_10825.jpg', 'real_27749.jpg', 'real_36635.jpg', 'real_60360.jpg', 'real_20781.jpg', 'real_11182.jpg', 'real_795.jpg', 'real_29225.jpg', 'real_7715.jpg', 'real_32695.jpg', 'real_27508.jpg', 'real_55830.jpg', 'real_59124.jpg', 'real_3780.jpg', 'real_10563.jpg', 'real_58184.jpg', 'real_246.jpg', 'real_66814.jpg', 'real_11147.jpg', 'real_47369.jpg', 'real_55296.jpg', 'real_12979.jpg', 'real_66987.jpg', 'real_6904.jpg', 'real_1578.jpg', 'real_55972.jpg', 'real_48827.jpg', 'real_4297.jpg', 'real_63753.jpg', 'real_41448.jpg', 'real_25255.jpg', 'real_41725.jpg', 'real_49321.jpg', 'real_36420.jpg', 'real_12375.jpg', 'real_28028.jpg', 'real_15642.jpg', 'real_18573.jpg', 'real_69585.jpg', 'real_4462.jpg', 'real_51595.jpg', 'real_56961.jpg', 'real_31175.jpg', 'real_30645.jpg', 'real_8833.jpg', 'real_31984.jpg', 'real_47080.jpg', 'real_42954.jpg', 'real_55831.jpg', 'real_58757.jpg', 'real_68169.jpg', 'real_7422.jpg', 'real_62033.jpg', 'real_67697.jpg', 'real_61232.jpg', 're

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory


In [ ]:
realPath="/content/Dataset/Train"
validatePath="/content/Dataset/Validation"

In [ ]:
image_paths = [os.path.join(realPath, fname) for fname in os.listdir(realPath) if fname.endswith('.jpg')]

In [ ]:
train_dataset = image_dataset_from_directory(
    realPath,
    image_size=(80,80),
    batch_size=32,
    label_mode="int"
)

Found 140002 files belonging to 2 classes.


In [ ]:
class_names = train_dataset.class_names
for idx, class_name in enumerate(class_names):
    print(f'Class {idx}: {class_name}')

Class 0: Fake
Class 1: Real


In [ ]:
def convert_to_grayscale(image, label):
    image = tf.image.rgb_to_grayscale(image)
    return image, label

# Apply the grayscale conversion to the dataset
train_dataset = train_dataset.map(convert_to_grayscale)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)


In [ ]:
for images, labels in train_dataset.take(1):
    print("Labels in the first batch:")
    print(labels.numpy())
    print(images.shape)

Labels in the first batch:
[0 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0]
(32, 120, 120, 3)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
model=Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(80,80,1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    layers.Dropout(0.2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 80, 80, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 78, 78, 64)        640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 39, 39, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 37, 37, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 18, 18, 128)       0         
 g2D)                                                            
                                                                 
 dropout (Dropout)           (None, 18, 18, 128)       0

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
res=model.fit(
    train_dataset,
    epochs=5
)

Epoch 1/5
4376/4376 [==============================] - 145s 24ms/step - loss: 0.2790 - accuracy: 0.8755
Epoch 2/5
4376/4376 [==============================] - 72s 16ms/step - loss: 0.1644 - accuracy: 0.9322
Epoch 3/5
4376/4376 [==============================] - 71s 16ms/step - loss: 0.1334 - accuracy: 0.9460
Epoch 4/5
4376/4376 [==============================] - 71s 16ms/step - loss: 0.1139 - accuracy: 0.9544
Epoch 5/5
4376/4376 [==============================] - 71s 16ms/step - loss: 0.0998 - accuracy: 0.9596


In [ ]:
val_dataset = image_dataset_from_directory(
    validatePath,
    image_size=(80,80),
    batch_size=32,
    label_mode="int"
)

Found 39428 files belonging to 2 classes.


In [ ]:
val_dataset = val_dataset.map(convert_to_grayscale)
for images, labels in val_dataset.take(1):
    print("Labels in the first batch:")
    print(labels.numpy())
    print(images.shape)

Labels in the first batch:
[0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0]
(32, 80, 80, 1)


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

val_dataset = val_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
model.evaluate(val_dataset)

1233/1233 [==============================] - 36s 9ms/step - loss: 0.1698 - accuracy: 0.9344


[0.16975045204162598, 0.9344374537467957]

In [ ]:
testPath="/content/Dataset/Test"
test_dataset = image_dataset_from_directory(
    testPath,
    image_size=(80,80),
    batch_size=32,
    label_mode="int"
  )
test_dataset=test_dataset.map(convert_to_grayscale)


Found 10905 files belonging to 2 classes.


In [ ]:
test_dataset=test_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [ ]:
model.evaluate(test_dataset)

341/341 [==============================] - 10s 6ms/step - loss: 0.2156 - accuracy: 0.9215


[0.21556620299816132, 0.9215039014816284]

In [ ]:
dir=os.listdir("/content/Dataset/Test/Fake")
dirList=[os.path.join("/content/Dataset/Test/Fake",x) for x in dir]

In [ ]:
print(dirList)

['/content/Dataset/Test/Real/real_795.jpg', '/content/Dataset/Test/Real/real_3780.jpg', '/content/Dataset/Test/Real/real_246.jpg', '/content/Dataset/Test/Real/real_1578.jpg', '/content/Dataset/Test/Real/real_4297.jpg', '/content/Dataset/Test/Real/real_4462.jpg', '/content/Dataset/Test/Real/real_4921.jpg', '/content/Dataset/Test/Real/real_4438.jpg', '/content/Dataset/Test/Real/real_1647.jpg', '/content/Dataset/Test/Real/real_1207.jpg', '/content/Dataset/Test/Real/real_230.jpg', '/content/Dataset/Test/Real/real_4974.jpg', '/content/Dataset/Test/Real/real_1484.jpg', '/content/Dataset/Test/Real/real_2513.jpg', '/content/Dataset/Test/Real/real_5289.jpg', '/content/Dataset/Test/Real/real_2580.jpg', '/content/Dataset/Test/Real/real_1239.jpg', '/content/Dataset/Test/Real/real_167.jpg', '/content/Dataset/Test/Real/real_3473.jpg', '/content/Dataset/Test/Real/real_2155.jpg', '/content/Dataset/Test/Real/real_1064.jpg', '/content/Dataset/Test/Real/real_4303.jpg', '/content/Dataset/Test/Real/real_52

In [ ]:
import cv2
for i in range(10):
  image = cv2.imread(dirList[i])
  grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  dimensions = (80,80)
  resized_image = cv2.resize(grayscale_image, dimensions)
  resized_image=resized_image.reshape(1,80,80,1)
  print(resized_image.shape)

  res=(model.predict(resized_image))
  if(res<0.5):
    print("Fake")
  else:
    print("Real")


(1, 80, 80, 1)
1/1 [==============================] - 0s 26ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 25ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 29ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 24ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 16ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 18ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 17ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 20ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 16ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 16ms/step
Fake


In [ ]:
model.save('/content/DeepFakeDetectionModel.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
TestModel = tf.keras.models.load_model('/content/DeepFakeDetectionModel.h5')

In [ ]:
import cv2
for i in range(10):
  image = cv2.imread(dirList[i])
  grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  dimensions = (80,80)
  resized_image = cv2.resize(grayscale_image, dimensions)
  resized_image=resized_image.reshape(1,80,80,1)
  print(resized_image.shape)

  res=(TestModel.predict(resized_image))
  if(res<0.5):
    print("Fake")
  else:
    print("Real")

(1, 80, 80, 1)
1/1 [==============================] - 0s 311ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 82ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 29ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 27ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 30ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 24ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 18ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 18ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 19ms/step
Fake
(1, 80, 80, 1)
1/1 [==============================] - 0s 19ms/step
Fake
